In [1]:
# Python Script
# https://www.electronicshub.org/raspberry-pi-l298n-interface-tutorial-control-dc-motor-l298n-raspberry-pi/


In [2]:
import RPi.GPIO as GPIO    
import json
from time import sleep

In [3]:
def get_bot_config(): 
    with open('autobots-pinout-config.json') as f:
        return json.load(f)

In [4]:
motor_frequency = {}
previous_motor_frequency = {}
in_pin = {}

bot_config = get_bot_config()

bot_config_frequency = bot_config['frequency'];
bot_config_key_mapping = bot_config['key-mapping'];

In [5]:
def init_gpio():
    GPIO.setmode(GPIO.BCM)
    
    for(k,v) in bot_config["wheels"].items():
        pin = {}
        for(k1,v1) in bot_config["sides-port"][v].items():
            pin[k1] = bot_config["pinout"][v1]
        init_gpio_sides(pin)
        motor_frequency[k] = init_pwm_motor_frequency(pin["en"])
        previous_motor_frequency[k] = -1
    init_motor_frequency()
    
def init_motor_frequency():
    for k in motor_frequency:
        motor_frequency[k].start(0)
        
def init_gpio_sides(pin_data):
    init_gpio_sides_with_data(pin_data["en"], pin_data["in-positive"], pin_data["in-negative"])
    
def init_gpio_sides_with_data(en, in_positive, in_negative):
   
    
    GPIO.setup(in_positive,GPIO.OUT)
    GPIO.setup(in_negative,GPIO.OUT)
    GPIO.setup(en,GPIO.OUT)
    
    GPIO.output(in_positive,GPIO.LOW)
    GPIO.output(in_negative,GPIO.LOW)
    
def init_pwm_motor_frequency(en):
    return GPIO.PWM(en, 1000)

In [6]:

def provide_input_for_bot(instructions):
    
    if instructions['f'] in bot_config_frequency:
        update_speed(instructions['f'])
    elif instructions['b'] in bot_config_frequency:
        update_speed(instructions['b'])
    else:
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["odd"]]["in-positive"]])
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["even"]]["in-positive"]])
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["odd"]]["in-negative"]])
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["even"]]["in-negative"]])
        
        
    if instructions['f'] != '0':
        high(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["odd"]]["in-positive"]])
        high(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["even"]]["in-positive"]])
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["odd"]]["in-negative"]])
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["even"]]["in-negative"]])
        
    if instructions['b'] != '0':
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["odd"]]["in-positive"]])
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["even"]]["in-positive"]])
        high(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["odd"]]["in-negative"]])
        high(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["even"]]["in-negative"]])
    
    if instructions['l'] == 1:
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["odd"]]["in-negative"]])
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["odd"]]["in-positive"]])
    
    if instructions['r'] == 1:
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["even"]]["in-negative"]])
        low(bot_config["pinout"][bot_config["sides-port"][bot_config["wheels"]["even"]]["in-positive"]])
        
    if 'e' in instructions and instructions['e'] == 1:
        clean_up_gpio()

In [7]:
def high(input_pin):
    GPIO.output(input_pin,GPIO.HIGH)
def low(input_pin):
    GPIO.output(input_pin,GPIO.LOW)

In [8]:

def update_speed(value):
    for(k,v) in bot_config["wheels"].items():    
        if previous_motor_frequency[k] != bot_config_frequency[value]:
            field = bot_config_frequency[value]
            motor_frequency[k].ChangeDutyCycle(field)
            previous_motor_frequency[k] = field


In [9]:
def clean_up_gpio():
    GPIO.cleanup()
    print("GPIO Clean up")

In [10]:
def demo_bot():
    init_gpio()
    provide_input_for_bot({"f":"0","b":"l","l":1,"r":0})

In [11]:
# demo_bot()